In [1]:
cd /home/tcechal/repos/temp/tmp-qt/src

/home/tcechal/repos/temp/tmp-qt/src


In [2]:
import simulation

from pipeline import Generator
from pipeline import Queue
from pipeline import Processor
from pipeline import Pipeline

from lib.generators import ConstantCountBatchProducer
from lib.generators import ConstantSizeWorkProducer
from lib.generators import ConstantDelayTimer

In [3]:
import plotly.offline 
from plotly.offline import download_plotlyjs
from plotly.offline import init_notebook_mode
import plotly.graph_objs

init_notebook_mode(connected=True)

In [4]:
utilization = 0.99

processor_slots = 1
processor_rate = 1.0
processing_rate = processor_slots * processor_rate

entry_rate = utilization * processing_rate

item_size = 1.0
batch_items = 1
batch_size = batch_items * item_size

total_batch_time = batch_size / entry_rate

delay_between_items = 0.0
delay_between_batches = total_batch_time - (batch_items - 1) * delay_between_items

In [5]:
generator = Generator(
    batch_producer=ConstantCountBatchProducer(count=batch_items),
    work_producer=ConstantSizeWorkProducer(size=item_size),
    batch_timer=ConstantDelayTimer(time=delay_between_batches),
    work_timer=ConstantDelayTimer(time=delay_between_items))

queue = Queue()
processor = Processor(slots=processor_slots, rate=processor_rate)

pipeline = Pipeline(generator, queue, processor)

In [6]:
max_time = 1000.0
simulation.run(pipeline, max_time)

In [7]:
params = {"t_min": 0.0, "t_max": max_time, "num_intervals": 100}
t1, y1 = queue.stats().mean_work_items_in_queue(**params)
t2, y2 = processor.stats().mean_processor_utilization(**params)

In [8]:
data = [
    plotly.graph_objs.Scatter(x=t1, y=y1, name="items in queue", line={'shape': 'hv'}),
    plotly.graph_objs.Scatter(x=t2, y=y2, name="processor utilization", line={'shape': 'hv'})
]
plotly.offline.iplot(data, filename='queue_utilization_timeline.html')

In [9]:
t, y = pipeline.sink.stats().total_batches_completed(**params)

data = [
    plotly.graph_objs.Scatter(x=t, y=y, name="batches completed", line={'shape': 'hv'}),
]
plotly.offline.iplot(data, filename='realized_value.html')

In [10]:
work_stats = pipeline.stats()

t, c = work_stats.item_cycle_time_histogram(time_resolution=1.0)

data = [plotly.graph_objs.Bar(x=t, y=c)]
plotly.offline.iplot(data, filename='cycle_time_distribution.html')

In [11]:
print("mean cycle time", work_stats.item_mean_cycle_time())

mean cycle time 1.0
